과제 1

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

def f(x):
    return x**4 -3*x**3 + 2

def fprime(x):
    h = 0.001
    return (f(x+h) - f(x))/h

all_x = tf.linspace(-2.0, 4.0, num=101)
all_y = f(all_x)

x = tf.Variable(4.0) # initial value
LR = 0.001

x_history, y_history = [], []

for _ in range(100):
  y = f(x)
  step = LR*fprime(x)

  x_history.append(x.numpy())
  y_history.append(y.numpy())

  x.assign_sub(step, read_value=False)  # x = x - lr*dx

fig, ax = plt.subplots(figsize=(5,5))
ax.plot(all_x, all_y, 'b-')
ax.scatter(x_history, y_history)
plt.plot(x_history, y_history, 'ro')
plt.xlabel('$x$')
plt.ylabel('$y=x^2-2x+1$')
plt.show()

과제 2

In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path='boston_housing.npz', test_split=0.2, seed=113
)
print('x_train.shape', 'y_train.shape', x_train.shape, y_train.shape)

all_train_data = np.hstack((x_train, y_train.reshape((-1, 1))))
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df = pd.DataFrame(all_train_data, columns=column_names)

# sns.pairplot(df[['CRIM', 'ZN', 'INDUS', 'TAX', 'MEDV']], diag_kind='kde')

x_mean = x_train.mean(axis=0)
x_std = x_train.std(axis=0)
x_train -= x_mean
x_train /= x_std
x_test -= x_mean
x_test /= x_std

y_mean = y_train.mean(axis=0)
y_std = y_train.std(axis=0)
y_train -= y_mean
y_train /= y_std
y_test -= y_mean
y_test /= y_std

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=52, activation='relu', input_shape=(13,)),
    tf.keras.layers.Dense(units=39, activation='relu'),
    tf.keras.layers.Dense(units=26, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.07), loss='mse')

@tf.function
def gradient(x):
    with tf.GradientTape() as tape:
        tape.watch(x)
        y = model(x)
    gradients = tape.gradient(y, x)
    return gradients

history = model.fit(x_train, y_train, epochs=25, batch_size=32, validation_split=0.25)

plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

model.evaluate(x_test, y_test)

pred_Y = model.predict(x_test)

plt.figure(figsize=(5, 5))
plt.plot(y_test, pred_Y, 'b.')
plt.axis([min(y_test), max(y_test), min(y_test), max(y_test)])
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], ls="--", c=".3")
plt.xlabel('test_Y')
plt.ylabel('pred_Y')
plt.show()
